# CPU Virtualization

The job of a CPU is to run the user programs.

We want to run many programs as once, N (# programs) >> M (# CPUs).

The OS makes it look like every program gets its own CPU (an illusion, but it definitley simplies things for the programs).

Key Abstraction: **The Process**

A **process** is a running program.

A process has:

- a set of CPU registers
  - general purpose registers for the program to use
  - specific registers that contain specific information about this process
    - **instruction pointer** aka **program counter**
    - memory address registers
- its own private memory address space

<img src="images/process_address_space.png" width="150">

This lecture will focus on the **mechanisms** of CPU virtualization. In future lectures, we will discuss **policies**.

How to virtualize the CPU:

To start simply, whenever we want to run a program, load it onto the CPU and just simply run it!

This is **Direct Execution**.

<img src="images/lde.png" width="500">

Three problems with the above simple scheme:

1) What if the process wants to perform a restricted operation?
2) What if the OS wants to switch processes, stop P_a, start P_b.
3) What if a process does something slow, like IO?